# Notebook 72: 計算グラフ ― 式をノードに変える

## Computation Graph: Transforming Equations into Nodes

---

### このノートブックの位置づけ

**Unit 0.0「ニューラルエンジンの深部」** の第3章として、連鎖律による微分計算を **自動化** するための **計算グラフ** を設計・実装します。

### 学習目標

1. 数式を **有向非巡回グラフ（DAG）** として表現する方法を理解する
2. Pythonで **Node クラス** を設計し、順伝播を実装する
3. 計算グラフの **トポロジカルソート** を理解する
4. 自動微分の準備として、グラフ構造で値を追跡する

### 前提知識

- Notebook 70-71 の内容（微分、連鎖律）
- Python のクラスと継承の基本

---

## 目次

1. [計算グラフとは](#1-計算グラフとは)
2. [基本ノードの設計](#2-基本ノードの設計)
3. [演算ノードの実装](#3-演算ノードの実装)
4. [グラフの構築と順伝播](#4-グラフの構築と順伝播)
5. [y = σ(Wx + b) を計算グラフで表現](#5-y--σwx--b-を計算グラフで表現)
6. [トポロジカルソート](#6-トポロジカルソート)
7. [可視化ツールの作成](#7-可視化ツールの作成)
8. [演習問題](#8-演習問題)
9. [まとめと次のステップ](#9-まとめと次のステップ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Circle, FancyArrowPatch
from abc import ABC, abstractmethod
from collections import deque
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

---

## 1. 計算グラフとは

### 1.1 数式からグラフへ

**計算グラフ（Computation Graph）** とは、数式を **ノード（演算）** と **エッジ（データの流れ）** で表現したものです。

例として、$y = (a + b) \times c$ を考えます：

```
    a ──┐
        ├── (+) ──┐
    b ──┘         ├── (×) ── y
                  │
    c ────────────┘
```

### 1.2 なぜ計算グラフが重要か

1. **逆伝播の自動化**: グラフ構造があれば、機械的に連鎖律を適用できる
2. **メモリ効率**: 必要な中間値だけを保持できる
3. **並列計算**: 依存関係が明確なので、並列化しやすい
4. **デバッグ**: 各ノードの値を検査できる

In [ ]:
# 計算グラフの概念図
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: 数式表記
ax = axes[0]
ax.axis('off')
ax.set_xlim(0, 10)
ax.set_ylim(0, 6)

ax.text(5, 5, '数式表記', ha='center', fontsize=14, fontweight='bold')
ax.text(5, 3.5, r'$y = (a + b) \times c$', ha='center', fontsize=20)
ax.text(5, 1.5, '・人間にとって読みやすい\n・計算順序が暗黙的', 
        ha='center', fontsize=11, color='gray')

# 右: 計算グラフ
ax = axes[1]
ax.axis('off')
ax.set_xlim(0, 10)
ax.set_ylim(0, 6)

ax.text(5, 5.5, '計算グラフ', ha='center', fontsize=14, fontweight='bold')

# ノード
nodes = {
    'a': (1, 4, 'a', 'lightblue'),
    'b': (1, 2, 'b', 'lightblue'),
    'c': (1, 0.5, 'c', 'lightblue'),
    '+': (4, 3, '+', 'lightgreen'),
    '×': (7, 2, '×', 'lightgreen'),
    'y': (9.5, 2, 'y', 'lightcoral'),
}

for name, (nx, ny, label, color) in nodes.items():
    circle = Circle((nx, ny), 0.5, facecolor=color, edgecolor='black', linewidth=2)
    ax.add_patch(circle)
    ax.text(nx, ny, label, ha='center', va='center', fontsize=14, fontweight='bold')

# エッジ
edges = [
    ('a', '+'), ('b', '+'), ('+', '×'), ('c', '×'), ('×', 'y')
]

for start, end in edges:
    sx, sy = nodes[start][0], nodes[start][1]
    ex, ey = nodes[end][0], nodes[end][1]
    ax.annotate('', xy=(ex - 0.5, ey), xytext=(sx + 0.5, sy),
                arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))

plt.suptitle(r'$y = (a + b) \times c$ の表現方法', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("【計算グラフの特徴】")
print("・各ノードは1つの演算を担当")
print("・エッジはデータの流れを表す")
print("・計算順序がグラフ構造から明確")

### 1.3 有向非巡回グラフ（DAG）

計算グラフは **DAG（Directed Acyclic Graph）** という構造を持ちます：

- **Directed（有向）**: エッジに方向がある（入力 → 出力）
- **Acyclic（非巡回）**: ループがない（出力が入力に戻らない）

DAGであることが重要な理由：
- 計算順序が一意に決まる
- 逆伝播が可能（ループがあると無限ループになる）

---

## 2. 基本ノードの設計

### 2.1 ノードの共通インターフェース

すべてのノードが持つべき機能：

1. **入力**: 親ノードからの値を受け取る
2. **出力**: 計算結果を子ノードへ渡す
3. **順伝播 (forward)**: 入力から出力を計算
4. **逆伝播 (backward)**: 勾配を計算して親ノードへ伝播（次のノートブック）

In [ ]:
class Node:
    """
    計算グラフのノードの基底クラス
    
    Attributes:
        value: 順伝播で計算された値
        grad: 逆伝播で計算された勾配
        inputs: 入力ノードのリスト
        name: デバッグ用の名前
    """
    
    _id_counter = 0  # ノードのユニークID生成用
    
    def __init__(self, name=None):
        self.value = None      # 順伝播の結果
        self.grad = None       # 逆伝播の勾配
        self.inputs = []       # 入力ノード
        self.outputs = []      # 出力ノード（逆伝播用）
        
        # ユニークIDと名前
        self.id = Node._id_counter
        Node._id_counter += 1
        self.name = name if name else f"Node_{self.id}"
    
    def forward(self):
        """
        順伝播: 入力から出力を計算
        サブクラスでオーバーライド
        """
        raise NotImplementedError("Subclass must implement forward()")
    
    def backward(self):
        """
        逆伝播: 勾配を計算して入力ノードへ伝播
        次のノートブックで実装
        """
        raise NotImplementedError("Subclass must implement backward()")
    
    def __repr__(self):
        return f"{self.__class__.__name__}(name='{self.name}', value={self.value})"


# テスト
node = Node(name="test")
print(f"ノード作成: {node}")
print(f"ID: {node.id}, 名前: {node.name}")

### 2.2 入力ノード（Variable）

計算グラフの **葉ノード** として、外部からの入力値を保持します。

In [ ]:
class Variable(Node):
    """
    入力変数ノード（計算グラフの葉）
    
    外部から値を受け取り、計算グラフの入力となる。
    学習対象のパラメータ（W, b）もこのクラスで表現する。
    """
    
    def __init__(self, value, name=None, requires_grad=True):
        """
        Args:
            value: 初期値（スカラーまたはnumpy配列）
            name: 変数名
            requires_grad: 勾配を計算するか（Trueなら学習対象）
        """
        super().__init__(name)
        self.value = np.atleast_1d(np.array(value, dtype=np.float64))
        self.requires_grad = requires_grad
        
        # 勾配の初期化
        if requires_grad:
            self.grad = np.zeros_like(self.value)
    
    def forward(self):
        """入力ノードは値をそのまま返す"""
        return self.value
    
    def set_value(self, value):
        """値を更新（学習時のパラメータ更新用）"""
        self.value = np.atleast_1d(np.array(value, dtype=np.float64))
        if self.requires_grad:
            self.grad = np.zeros_like(self.value)
    
    def __repr__(self):
        return f"Variable('{self.name}', value={self.value.item() if self.value.size == 1 else self.value})"


# テスト
x = Variable(3.0, name='x')
W = Variable(0.5, name='W')
b = Variable(-0.2, name='b')

print("入力変数の作成:")
print(f"  {x}")
print(f"  {W}")
print(f"  {b}")

---

## 3. 演算ノードの実装

### 3.1 演算ノードの基底クラス

In [ ]:
class Operation(Node):
    """
    演算ノードの基底クラス
    
    入力ノードを受け取り、何らかの演算を行い、結果を出力する。
    """
    
    def __init__(self, *inputs, name=None):
        """
        Args:
            *inputs: 入力ノード（可変長引数）
            name: 演算の名前
        """
        super().__init__(name)
        self.inputs = list(inputs)
        
        # 入力ノードの出力リストに自分を追加（逆伝播用）
        for inp in self.inputs:
            inp.outputs.append(self)
        
        # 勾配の初期化
        self.grad = None
    
    def _get_input_values(self):
        """入力ノードの値を取得"""
        return [inp.value for inp in self.inputs]


print("Operation基底クラスを定義しました")

### 3.2 加算ノード（Add）

In [ ]:
class Add(Operation):
    """
    加算演算: z = x + y
    
    順伝播: z = x + y
    逆伝播: ∂L/∂x = ∂L/∂z, ∂L/∂y = ∂L/∂z
            （勾配をそのまま両方に分配）
    """
    
    def __init__(self, x, y, name=None):
        super().__init__(x, y, name=name or 'add')
    
    def forward(self):
        """順伝播: z = x + y"""
        x_val, y_val = self._get_input_values()
        self.value = x_val + y_val
        return self.value
    
    def backward(self):
        """逆伝播: 勾配をそのまま入力に分配"""
        # ∂L/∂x = ∂L/∂z × ∂z/∂x = ∂L/∂z × 1 = ∂L/∂z
        # ∂L/∂y = ∂L/∂z × ∂z/∂y = ∂L/∂z × 1 = ∂L/∂z
        x, y = self.inputs
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        if y.grad is None:
            y.grad = np.zeros_like(y.value)
        
        x.grad = x.grad + self.grad  # 勾配の加算（分岐がある場合）
        y.grad = y.grad + self.grad


# テスト
a = Variable(2.0, name='a')
b = Variable(3.0, name='b')
add_node = Add(a, b, name='a+b')

result = add_node.forward()
print(f"加算テスト: {a.value.item()} + {b.value.item()} = {result.item()}")

### 3.3 乗算ノード（Multiply）

In [ ]:
class Multiply(Operation):
    """
    乗算演算: z = x × y
    
    順伝播: z = x × y
    逆伝播: ∂L/∂x = ∂L/∂z × y, ∂L/∂y = ∂L/∂z × x
            （勾配に相手の値を掛ける）
    """
    
    def __init__(self, x, y, name=None):
        super().__init__(x, y, name=name or 'mul')
    
    def forward(self):
        """順伝播: z = x × y"""
        x_val, y_val = self._get_input_values()
        self.value = x_val * y_val
        return self.value
    
    def backward(self):
        """逆伝播: 勾配に相手の値を掛けて入力に分配"""
        x, y = self.inputs
        x_val, y_val = self._get_input_values()
        
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        if y.grad is None:
            y.grad = np.zeros_like(y.value)
        
        # ∂L/∂x = ∂L/∂z × y
        # ∂L/∂y = ∂L/∂z × x
        x.grad = x.grad + self.grad * y_val
        y.grad = y.grad + self.grad * x_val


# テスト
a = Variable(4.0, name='a')
b = Variable(5.0, name='b')
mul_node = Multiply(a, b, name='a×b')

result = mul_node.forward()
print(f"乗算テスト: {a.value.item()} × {b.value.item()} = {result.item()}")

### 3.4 シグモイドノード（Sigmoid）

In [ ]:
class Sigmoid(Operation):
    """
    シグモイド活性化関数: y = σ(x) = 1 / (1 + exp(-x))
    
    順伝播: y = σ(x)
    逆伝播: ∂L/∂x = ∂L/∂y × σ(x)(1 - σ(x))
    """
    
    def __init__(self, x, name=None):
        super().__init__(x, name=name or 'sigmoid')
    
    def forward(self):
        """順伝播: y = 1 / (1 + exp(-x))"""
        x_val = self.inputs[0].value
        # 数値安定性のための実装
        self.value = np.where(
            x_val >= 0,
            1 / (1 + np.exp(-x_val)),
            np.exp(x_val) / (1 + np.exp(x_val))
        )
        return self.value
    
    def backward(self):
        """逆伝播: ∂L/∂x = ∂L/∂y × σ(x)(1 - σ(x))"""
        x = self.inputs[0]
        
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        
        # σ'(x) = σ(x)(1 - σ(x))
        sigmoid_derivative = self.value * (1 - self.value)
        x.grad = x.grad + self.grad * sigmoid_derivative


# テスト
z = Variable(0.0, name='z')
sigmoid_node = Sigmoid(z, name='σ(z)')

result = sigmoid_node.forward()
print(f"シグモイドテスト: σ({z.value.item()}) = {result.item()}")
print(f"（σ(0) = 0.5 であることを確認）")

### 3.5 その他の基本演算ノード

In [ ]:
class ReLU(Operation):
    """
    ReLU活性化関数: y = max(0, x)
    
    順伝播: y = max(0, x)
    逆伝播: ∂L/∂x = ∂L/∂y × (1 if x > 0 else 0)
    """
    
    def __init__(self, x, name=None):
        super().__init__(x, name=name or 'relu')
        self._mask = None  # x > 0 のマスク（逆伝播用）
    
    def forward(self):
        x_val = self.inputs[0].value
        self._mask = (x_val > 0).astype(float)
        self.value = np.maximum(0, x_val)
        return self.value
    
    def backward(self):
        x = self.inputs[0]
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        x.grad = x.grad + self.grad * self._mask


class Square(Operation):
    """
    二乗演算: y = x²
    
    順伝播: y = x²
    逆伝播: ∂L/∂x = ∂L/∂y × 2x
    """
    
    def __init__(self, x, name=None):
        super().__init__(x, name=name or 'square')
    
    def forward(self):
        x_val = self.inputs[0].value
        self.value = x_val ** 2
        return self.value
    
    def backward(self):
        x = self.inputs[0]
        x_val = x.value
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        x.grad = x.grad + self.grad * 2 * x_val


class ScalarMultiply(Operation):
    """
    スカラー倍: y = c × x（cは定数）
    """
    
    def __init__(self, x, scalar, name=None):
        super().__init__(x, name=name or f'scale({scalar})')
        self.scalar = scalar
    
    def forward(self):
        x_val = self.inputs[0].value
        self.value = self.scalar * x_val
        return self.value
    
    def backward(self):
        x = self.inputs[0]
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        x.grad = x.grad + self.grad * self.scalar


# テスト
print("その他のノードのテスト:")

x = Variable(-2.0, name='x')
relu_node = ReLU(x)
print(f"  ReLU(-2) = {relu_node.forward().item()}")

x = Variable(3.0, name='x')
square_node = Square(x)
print(f"  Square(3) = {square_node.forward().item()}")

---

## 4. グラフの構築と順伝播

### 4.1 $y = (a + b) \times c$ を計算グラフで構築

In [ ]:
# y = (a + b) × c を計算グラフで表現

# 入力変数の定義
a = Variable(2.0, name='a')
b = Variable(3.0, name='b')
c = Variable(4.0, name='c')

# 計算グラフの構築
# Step 1: t = a + b
t = Add(a, b, name='t=a+b')

# Step 2: y = t × c
y = Multiply(t, c, name='y=t×c')

# 順伝播の実行
print("【計算グラフ: y = (a + b) × c】\n")
print("順伝播:")

t_val = t.forward()
print(f"  Step 1: t = a + b = {a.value.item()} + {b.value.item()} = {t_val.item()}")

y_val = y.forward()
print(f"  Step 2: y = t × c = {t_val.item()} × {c.value.item()} = {y_val.item()}")

print(f"\n結果: y = {y_val.item()}")

# 検算
direct = (2.0 + 3.0) * 4.0
print(f"検算: (2 + 3) × 4 = {direct}")

### 4.2 計算グラフの構造を確認

In [ ]:
def print_graph_structure(output_node, indent=0):
    """計算グラフの構造を再帰的に表示"""
    prefix = "  " * indent
    node_type = output_node.__class__.__name__
    value_str = f"{output_node.value.item():.4f}" if output_node.value is not None and output_node.value.size == 1 else str(output_node.value)
    
    print(f"{prefix}{node_type}('{output_node.name}'): value = {value_str}")
    
    if hasattr(output_node, 'inputs'):
        for inp in output_node.inputs:
            print_graph_structure(inp, indent + 1)


print("計算グラフの構造（出力から入力へ遡る）:\n")
print_graph_structure(y)

---

## 5. y = σ(Wx + b) を計算グラフで表現

ニューラルネットワークの基本単位を計算グラフで構築します。

In [ ]:
def build_neuron_graph(x_val, W_val, b_val):
    """
    y = σ(Wx + b) の計算グラフを構築
    
    計算グラフ:
        x ─┬─ (×) ─┬─ (+) ─── (σ) ─── y
           │       │
        W ─┘       b
    """
    # 入力変数
    x = Variable(x_val, name='x', requires_grad=False)  # 入力データ（勾配不要）
    W = Variable(W_val, name='W', requires_grad=True)   # 重み（学習対象）
    b = Variable(b_val, name='b', requires_grad=True)   # バイアス（学習対象）
    
    # 計算グラフの構築
    # Step 1: z1 = W × x
    z1 = Multiply(W, x, name='z1=Wx')
    
    # Step 2: z2 = z1 + b
    z2 = Add(z1, b, name='z2=z1+b')
    
    # Step 3: y = σ(z2)
    y = Sigmoid(z2, name='y=σ(z2)')
    
    return {'x': x, 'W': W, 'b': b, 'z1': z1, 'z2': z2, 'y': y}


# グラフの構築
graph = build_neuron_graph(x_val=2.0, W_val=0.5, b_val=-0.3)

print("【y = σ(Wx + b) の計算グラフ構築】\n")
print(f"入力パラメータ:")
print(f"  x = {graph['x'].value.item()}")
print(f"  W = {graph['W'].value.item()}")
print(f"  b = {graph['b'].value.item()}")

In [ ]:
# 順伝播の実行
print("\n順伝播の実行:\n")

# Step 1
z1_val = graph['z1'].forward()
print(f"Step 1: z1 = W × x = {graph['W'].value.item()} × {graph['x'].value.item()} = {z1_val.item()}")

# Step 2
z2_val = graph['z2'].forward()
print(f"Step 2: z2 = z1 + b = {z1_val.item()} + {graph['b'].value.item()} = {z2_val.item()}")

# Step 3
y_val = graph['y'].forward()
print(f"Step 3: y = σ(z2) = σ({z2_val.item()}) = {y_val.item():.6f}")

# 検算
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

x, W, b = 2.0, 0.5, -0.3
direct = sigmoid(W * x + b)
print(f"\n検算: σ({W} × {x} + {b}) = σ({W * x + b}) = {direct:.6f}")

In [ ]:
# 計算グラフの可視化
fig, ax = plt.subplots(figsize=(14, 5))
ax.set_xlim(0, 14)
ax.set_ylim(0, 4)
ax.axis('off')

# ノード位置
node_positions = {
    'x': (1, 3),
    'W': (1, 1),
    'b': (5, 1),
    '×': (3, 2),
    '+': (6, 2),
    'σ': (9, 2),
    'y': (12, 2),
}

# ノードの描画
node_styles = {
    'x': ('x\n=2.0', 'lightblue'),
    'W': ('W\n=0.5', 'lightyellow'),
    'b': ('b\n=-0.3', 'lightyellow'),
    '×': ('×', 'lightgreen'),
    '+': ('+', 'lightgreen'),
    'σ': ('σ', 'lightgreen'),
    'y': ('y\n=0.646', 'lightcoral'),
}

for name, (nx, ny) in node_positions.items():
    label, color = node_styles[name]
    circle = Circle((nx, ny), 0.6, facecolor=color, edgecolor='black', linewidth=2)
    ax.add_patch(circle)
    ax.text(nx, ny, label, ha='center', va='center', fontsize=10)

# エッジの描画
edges = [
    ('x', '×', ''),
    ('W', '×', ''),
    ('×', '+', 'z1=1.0'),
    ('b', '+', ''),
    ('+', 'σ', 'z2=0.7'),
    ('σ', 'y', ''),
]

for start, end, label in edges:
    sx, sy = node_positions[start]
    ex, ey = node_positions[end]
    ax.annotate('', xy=(ex - 0.65, ey), xytext=(sx + 0.65, sy),
                arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))
    if label:
        mx, my = (sx + ex) / 2, (sy + ey) / 2 + 0.4
        ax.text(mx, my, label, ha='center', fontsize=9, color='blue')

ax.set_title(r'計算グラフ: $y = \sigma(Wx + b)$', fontsize=14)

# 凡例
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightblue', markersize=15, label='入力データ'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightyellow', markersize=15, label='学習パラメータ'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightgreen', markersize=15, label='演算'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightcoral', markersize=15, label='出力'),
]
ax.legend(handles=legend_elements, loc='upper right', fontsize=9)

plt.tight_layout()
plt.show()

---

## 6. トポロジカルソート

### 6.1 なぜ計算順序が重要か

計算グラフのノードは **依存関係** を持っています。例えば：
- `z1 = W × x` は `W` と `x` の値が決まってから計算できる
- `z2 = z1 + b` は `z1` と `b` の値が決まってから計算できる

**トポロジカルソート** は、この依存関係を満たす計算順序を決定するアルゴリズムです。

In [ ]:
def topological_sort(output_node):
    """
    計算グラフをトポロジカルソートして、順伝播の順序を決定
    
    Args:
        output_node: 計算グラフの出力ノード
    
    Returns:
        順伝播すべきノードのリスト（依存関係を満たす順序）
    """
    visited = set()
    order = []
    
    def dfs(node):
        if id(node) in visited:
            return
        visited.add(id(node))
        
        # まず入力ノードを処理（依存関係）
        if hasattr(node, 'inputs'):
            for inp in node.inputs:
                dfs(inp)
        
        # 入力が全て処理されたら自分を追加
        order.append(node)
    
    dfs(output_node)
    return order


# テスト
sorted_nodes = topological_sort(graph['y'])

print("トポロジカルソートの結果（順伝播の順序）:\n")
for i, node in enumerate(sorted_nodes):
    node_type = node.__class__.__name__
    print(f"  {i+1}. {node_type}('{node.name}')")

### 6.2 自動順伝播の実装

In [ ]:
def forward_pass(output_node, verbose=False):
    """
    計算グラフの自動順伝播
    
    Args:
        output_node: 計算グラフの出力ノード
        verbose: 詳細出力するか
    
    Returns:
        出力ノードの値
    """
    # トポロジカルソートで順序を決定
    sorted_nodes = topological_sort(output_node)
    
    if verbose:
        print("自動順伝播:\n")
    
    # 順番に順伝播を実行
    for node in sorted_nodes:
        result = node.forward()
        if verbose:
            value_str = f"{result.item():.6f}" if result.size == 1 else str(result)
            print(f"  {node.name}: {value_str}")
    
    return output_node.value


# 新しいグラフで自動順伝播をテスト
graph2 = build_neuron_graph(x_val=1.5, W_val=0.8, b_val=0.1)
result = forward_pass(graph2['y'], verbose=True)

print(f"\n最終出力: y = {result.item():.6f}")

---

## 7. 可視化ツールの作成

計算グラフをより直感的に理解するための可視化ツールを作成します。

In [ ]:
def visualize_computation_graph(output_node, title="Computation Graph"):
    """
    計算グラフを自動的に可視化
    """
    # ノードを収集
    sorted_nodes = topological_sort(output_node)
    
    # ノードの層（深さ）を計算
    depth = {}
    for node in sorted_nodes:
        if isinstance(node, Variable):
            depth[id(node)] = 0
        else:
            max_input_depth = max(depth[id(inp)] for inp in node.inputs)
            depth[id(node)] = max_input_depth + 1
    
    # 層ごとにノードをグループ化
    max_depth = max(depth.values())
    layers = [[] for _ in range(max_depth + 1)]
    for node in sorted_nodes:
        layers[depth[id(node)]].append(node)
    
    # 位置を計算
    positions = {}
    for d, layer_nodes in enumerate(layers):
        n_nodes = len(layer_nodes)
        for i, node in enumerate(layer_nodes):
            x = d * 2 + 1
            y = (n_nodes - 1) / 2 - i + 1.5
            positions[id(node)] = (x, y)
    
    # 描画
    fig, ax = plt.subplots(figsize=(max_depth * 3 + 2, max(len(l) for l in layers) * 1.5 + 1))
    ax.set_xlim(-0.5, max_depth * 2 + 2.5)
    ax.set_ylim(-0.5, max(len(l) for l in layers) + 1)
    ax.axis('off')
    
    # ノードの色
    def get_node_color(node):
        if isinstance(node, Variable):
            return 'lightyellow' if node.requires_grad else 'lightblue'
        return 'lightgreen'
    
    # ノードを描画
    for node in sorted_nodes:
        nx, ny = positions[id(node)]
        color = get_node_color(node)
        
        circle = Circle((nx, ny), 0.4, facecolor=color, edgecolor='black', linewidth=1.5)
        ax.add_patch(circle)
        
        # ラベル
        label = node.name.split('=')[0] if '=' in node.name else node.name
        if len(label) > 6:
            label = label[:5] + '..'
        ax.text(nx, ny, label, ha='center', va='center', fontsize=8)
        
        # 値
        if node.value is not None:
            val_str = f"{node.value.item():.3f}" if node.value.size == 1 else "arr"
            ax.text(nx, ny - 0.6, val_str, ha='center', va='center', fontsize=7, color='blue')
    
    # エッジを描画
    for node in sorted_nodes:
        if hasattr(node, 'inputs'):
            ex, ey = positions[id(node)]
            for inp in node.inputs:
                sx, sy = positions[id(inp)]
                ax.annotate('', xy=(ex - 0.4, ey), xytext=(sx + 0.4, sy),
                            arrowprops=dict(arrowstyle='->', color='gray', lw=1))
    
    ax.set_title(title, fontsize=12)
    plt.tight_layout()
    return fig, ax


# グラフの可視化
visualize_computation_graph(graph['y'], title=r"$y = \sigma(Wx + b)$ の計算グラフ")
plt.show()

---

## 8. 演習問題

### 演習 8.1: より複雑な計算グラフ

次の式を計算グラフで表現し、順伝播を実行してください：

$$
y = (x_1 \cdot w_1 + x_2 \cdot w_2 + b)^2
$$

In [ ]:
# 演習 8.1: 解答欄

def build_two_input_graph(x1_val, x2_val, w1_val, w2_val, b_val):
    """
    y = (x1*w1 + x2*w2 + b)² の計算グラフを構築
    
    TODO: 以下を実装
    1. 入力変数の定義
    2. 計算グラフの構築
    3. 辞書で返す
    """
    pass


# テストコード（実装後にコメントを外して実行）
# graph = build_two_input_graph(1.0, 2.0, 0.5, -0.3, 0.1)
# result = forward_pass(graph['y'], verbose=True)
# 
# # 検算
# expected = (1.0 * 0.5 + 2.0 * (-0.3) + 0.1) ** 2
# print(f"\n検算: (1.0×0.5 + 2.0×(-0.3) + 0.1)² = {expected:.6f}")

### 演習 8.2: 新しい演算ノードの追加

`Exp` ノード（$y = e^x$）を実装してください。

In [ ]:
# 演習 8.2: 解答欄

class Exp(Operation):
    """
    指数関数: y = exp(x)
    
    TODO:
    - forward(): y = exp(x) を計算
    - backward(): ∂L/∂x = ∂L/∂y × exp(x) を計算
    """
    
    def __init__(self, x, name=None):
        super().__init__(x, name=name or 'exp')
    
    def forward(self):
        # TODO: 実装
        pass
    
    def backward(self):
        # TODO: 実装
        pass


# テストコード（実装後にコメントを外して実行）
# x = Variable(1.0, name='x')
# exp_node = Exp(x)
# result = exp_node.forward()
# print(f"exp(1.0) = {result.item():.6f}")
# print(f"検算: e^1 = {np.exp(1):.6f}")

### 演習 8.3: ComputationGraph クラスの作成

グラフ全体を管理するクラスを設計してください。

In [ ]:
# 演習 8.3: 解答欄

class ComputationGraph:
    """
    計算グラフ全体を管理するクラス
    
    機能:
    - ノードの登録
    - 順伝播の自動実行
    - グラフの可視化
    - パラメータの取得（学習対象のVariable）
    """
    
    def __init__(self):
        self.nodes = []       # 全ノード
        self.output = None    # 出力ノード
    
    def add_variable(self, value, name=None, requires_grad=True):
        """入力変数を追加"""
        # TODO: 実装
        pass
    
    def forward(self):
        """順伝播を実行"""
        # TODO: 実装
        pass
    
    def get_parameters(self):
        """学習対象のパラメータを取得"""
        # TODO: 実装
        pass


# テストコード

---

## 9. まとめと次のステップ

### このノートブックで学んだこと

1. **計算グラフの概念**: 数式を **ノード（演算）** と **エッジ（データフロー）** で表現

2. **ノードの設計**:
   - `Variable`: 入力変数（葉ノード）
   - `Operation`: 演算ノード（内部ノード）
   - 共通インターフェース: `forward()`, `backward()`

3. **基本演算ノード**:
   - `Add`: 加算（勾配をそのまま分配）
   - `Multiply`: 乗算（勾配に相手の値を掛ける）
   - `Sigmoid`: シグモイド活性化
   - `ReLU`: ReLU活性化

4. **トポロジカルソート**: 依存関係を満たす計算順序の決定

5. **自動順伝播**: グラフ構造を使って計算を自動化

### 次のノートブック（73: 逆伝播の誕生）への橋渡し

このノートブックでは順伝播を完成させました。次のノートブックでは：

- 各ノードの `backward()` メソッドを本格的に実装
- **勾配の逆流** を計算グラフ上で実現
- 損失関数を追加して **完全な学習ループ** の準備

を行います。

---

## 付録: ノードクラスの一覧

```python
# 基底クラス
class Node:          # すべてのノードの基底
class Variable(Node): # 入力変数（葉ノード）
class Operation(Node): # 演算ノードの基底

# 演算ノード
class Add(Operation):      # 加算: z = x + y
class Multiply(Operation): # 乗算: z = x × y
class Sigmoid(Operation):  # シグモイド: y = σ(x)
class ReLU(Operation):     # ReLU: y = max(0, x)
class Square(Operation):   # 二乗: y = x²
class ScalarMultiply(Operation): # スカラー倍: y = c × x
```